## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt

In [2]:
import Portfolio as port # This is from the Portfolio.py file with the different classes defined

## 1. Set-up Portfolio Strategy

In [3]:
# Define start date and end date

start_date = '2018-01-01'

end_date = '2020-01-01'

In [4]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

voo = port.Share('VOO', 'Equity')
bnd = port.Share('BND', 'Bond')

In [5]:
voo.get_value(start_date)

247.09

In [6]:
voo.get_value('2019-10-31')

278.55

In [12]:
voo.div

,action,value
2019-12-23,DIVIDEND,1.429
2019-09-26,DIVIDEND,1.301
2019-06-27,DIVIDEND,1.386
2019-03-21,DIVIDEND,1.455
2018-12-17,DIVIDEND,1.289
2018-09-26,DIVIDEND,1.207
2018-06-28,DIVIDEND,1.157
2018-03-26,DIVIDEND,1.084
2017-12-26,DIVIDEND,1.184
2017-09-20,DIVIDEND,1.176


In [7]:
# Need a way to group the different shares together

shares_list = [voo, bnd]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type


In [8]:
shares_dict

{<Portfolio.Share at 0x7f68c8a81a58>: 'Equity',
 <Portfolio.Share at 0x7f68c8a817b8>: 'Bond'}

In [9]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(50,50,0,5)

In [10]:
# Run portfolio with set start and end dates

portfolio = port.Portfolio(shares_dict)

portfolio.initial_buy(500, strat, start_date)

In [13]:
# Run the portfolio over a series of months

time_period = pd.date_range(pd.to_datetime('2018-01-01'),pd.to_datetime('2018-04-01'))

for day in time_period:
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    

{'Equities': 250.0, 'Bonds': 250.17960553892215, 'Cash': 0}
{'Equities': 250.0, 'Bonds': 250.17960553892215, 'Cash': 0}
{'Equities': 251.48731231535066, 'Bonds': 250.39490640427405, 'Cash': 0}
{'Equities': 252.59014933829778, 'Bonds': 250.27187733835868, 'Cash': 0}
{'Equities': 254.2089926747339, 'Bonds': 249.9335474070914, 'Cash': 0}
{'Equities': 254.73511675907565, 'Bonds': 249.9643046735702, 'Cash': 0}
{'Equities': 254.73511675907565, 'Bonds': 249.9643046735702, 'Cash': 0}
{'Equities': 254.73511675907565, 'Bonds': 249.9643046735702, 'Cash': 0}
{'Equities': 255.2713586142701, 'Bonds': 249.22613027807796, 'Cash': 0}
{'Equities': 254.8362944676029, 'Bonds': 249.25688754455683, 'Cash': 0}
{'Equities': 256.71819984620987, 'Bonds': 249.25688754455683, 'Cash': 0}
{'Equities': 258.4179853494678, 'Bonds': 249.22613027807796, 'Cash': 0}
{'Equities': 257.4770326601643, 'Bonds': 249.41067387695105, 'Cash': 0}
{'Equities': 257.4770326601643, 'Bonds': 249.41067387695105, 'Cash': 0}
{'Equities': 2

In [14]:
portfolio.log

{0: ['Buy',
  250.0,
  247.09,
  1.0117770852725727,
  '2018-01-01'],
 1: ['Buy',
  250.0,
  81.34,
  3.0735185640521268,
  '2018-01-01'],
 2: ['Buy',
  0.177,
  80.16,
  0.002208083832335329,
  Timestamp('2018-02-01 00:00:00', freq='D')],
 3: ['Buy',
  0.177,
  80.16,
  0.002208083832335329,
  Timestamp('2018-02-01 00:00:00', freq='D')],
 4: ['Buy',
  0.165,
  79.52,
  0.002074949698189135,
  Timestamp('2018-03-01 00:00:00', freq='D')],
 5: ['Buy',
  1.084,
  243.53,
  0.004451196977785078,
  Timestamp('2018-03-26 00:00:00', freq='D')]}

In [ ]:
time_period

In [ ]:
start_date

In [ ]:
# Get the value a few days later

portfolio.get_value('2018-05-01')

In [ ]:
portfolio.get_asset_values('2018-05-01')

In [ ]:
portfolio.asset_values

In [ ]:
for share in portfolio.shares:
    print(share.div.index)

In [ ]:
portfolio.reinvest_divs('2018-05-01')

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas_datareader

In [ ]:
## This is useful:
#https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
bnd = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp = web.DataReader('VOO', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp.head(3)

In [ ]:
bnd.head(3)

In [ ]:
bnd.loc['2017-07-07']['High']

In [ ]:
# Import dividend records
bnd_div = web.DataReader('BND', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')
sp_div = web.DataReader('VOO', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')

In [ ]:
bnd_div.head(3)

In [ ]:
sp_div.head(3)

In [ ]:
sp_div.value.loc['2020-09-29']

In [ ]:
sp_div[sp_div.index < pd.to_datetime('2016-01-01')]

In [ ]:
sp_div.index

In [ ]:
str('sp')+str('_div')

In [ ]:
Assets = {}
Assets['Bonds'] = bnd
Assets['Equities'] = sp

In [ ]:
Dividends

In [ ]:
# Maybe it is worth merging the dividend and share price files together for easier reference?
# Or build a dictionary to allow easy mapping?

## 1. Calculate Value Given a Start and End Date

In [ ]:
start_date = '2015-11-20'
end_date = 

In [ ]:
def calc_value(share, divs, start, end, initial_value):
    # Take the average of the day's high and low for starting share price
    num_shares = initial_value/np.mean([share.loc[start]['Low'],share.loc[start]['High']])
    divs = divs[(divs.index <= pd.to_datetime(end)) & (divs.index >= pd.to_datetime(start))]
    for date in divs.index:
        price = np.mean([share.High.loc[date], share.Low.loc[date]])
        div_value = divs.value.loc[date]
        num_shares += div_value/price
    output = {}
    output['Value'] = num_shares*np.mean([share.loc[end]['Low'],share.loc[end]['High']])
    output['ROI'] = output['Value']-initial_value
    return(output)
    
    # Will need to factor in other actions later, but this will work for now
    
    


In [ ]:
calc_value(sp,sp_div, '2015-11-20', '2017-11-20',100)

In [ ]:
calc_value(bnd,bnd_div, '2015-11-20', '2017-11-20',100)

In [ ]:
sp.Close.plot()

In [ ]:
bnd.Close.plot()

In [ ]:
# Identify the ROI with different asset distributions

In [ ]:
class Share:

    def __init__(self, name, asset_type):
        """Initialize attributes."""
        self.name = name
        self.type = asset_type
        #self.amount = amount
        
    def get_value(self, date):
        df = web.DataReader(self.name, 'yahoo', start=date, end=date)
        return round(df.loc[date,'Close'],2)
    

In [ ]:
bnd = Share('BND','Bond')

In [ ]:
sp = Share('VOO','Equity')

In [ ]:
class Strategy:
    
    def __init__(self,equity_distribution, bond_distribution, cash_distribution, threshold):
        self.equity_distribution = equity_distribution
        self.bond_distribution = bond_distribution
        self.cash_distribution = 100-(equity_distribution+bond_distribution)
        self.threshold = threshold
    

In [ ]:
class Portfolio:
    
    def __init__(self):
        self.shares = {}
        self.log = {}
        self.cash_bal = 0
        self.asset_split = {'Equities': None,'Bonds': None,'Cash': None}
        self.asset_values = {'Equities': None,'Bonds': None,'Cash': None}
        
    def buy(self, share, amount, date):
        purchase_price = share.get_value(date)
        shares = amount/purchase_price
        # Record the transaction
        self.log[len(self.log)] = ['Buy',share,amount,purchase_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] += shares
        elif share not in self.shares:
            self.shares[share] = shares
            
    def sell(self, share, amount, date):
        sell_price = share.get_value(date)
        shares = amount/sell_price
        # Record the transaction
        self.log[len(self.log)] = ['Sell',share,amount,sell_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] -= shares
        elif share not in self.shares:
            self.shares[share] = shares
        cash_bal += amount
        
            
    def get_value(self,date):
        value = self.cash_bal
        # Iterate through the 'shares' attribute and get the current value for each 'Share' object
        for share in self.shares:
            value += share.get_value(date)*self.shares[share]
        return(value)
    
    def get_asset_values(self,date):
        bond_val = 0
        eq_val = 0
        for share in self.shares:
            if share.type == 'Bond':
                bond_val += share.get_value(date)*self.shares[share]
            elif share.type == 'Equity':
                eq_val += share.get_value(date)*self.shares[share]
        total = bond_val+eq_val+self.cash_bal
        self.asset_values = {'Equities': eq_val,'Bonds': bond_val,'Cash': self.cash_bal}
        self.asset_split = {'Equities': (eq_val/total)*100,'Bonds': (bond_val/total)*100,'Cash': (self.cash_bal/total)*100}
            
        
    

In [ ]:
portfolio = Portfolio()

In [ ]:
portfolio.get_value)

In [ ]:
# Calculate portfolio value

start_date = '2015-11-20'
end_date = '2020-11-20'

portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

portfolio.get_value(end_date)

In [ ]:
bond = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
dates = list(bond.index)

In [ ]:
dates = [x.strftime('%Y-%m-%d') for x in dates]

In [ ]:
# Deploy balancing strategy

strategy = Strategy(75,25,0,1)

# Initial Purchase (manually allocated proportions)
portfolio = Portfolio() 
portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

for date in dates:
    portfolio.get_asset_values(date)
    if portfolio.asset_split['Equities'] > strategy.equity_distribution+strategy.threshold:
        amount_to_sell = (portfolio.asset_split['Equities']-strategy.equity_distribution)*portfolio.asset_values['Equities']
        portfolio.sell(sp,amount_to_sell,date)
        portfolio.buy(bnd,amount_to_sell,date)
        

    

In [ ]:
portfolio.get_asset_values('2019-10-29')

In [ ]:
portfolio.asset_split['Equities']